# 竞价撮合
https://www.fmz.com/bbs-topic/560

In [87]:
# 买/卖委托单
class Order:
    current_order_id = 1 # 唯一订单ID分配
    
    BUY = 'buy' # 买单 
    SELL = 'sell' # 卖单
    
    def __init__(self, order_type, order_amount, order_price):
        self.id = Order.current_order_id
        Order.current_order_id += 1
        self.type = order_type
        self.amount = order_amount
        self.price = order_price
        
    def __str__(self):
        return '订单ID[%5d] 操作类型[%4s] 数量[%5d] 单价[%.02f]' % (
            self.id, self.type, self.amount, self.price)

In [88]:
# 成交的交易
class Trade:
    current_trade_id = 1
    
    def __init__(self, buy_order_id, sell_order_id, trade_amount, buy_price, sell_price, trade_price):
        self.id = Trade.current_trade_id
        Trade.current_trade_id += 1
        self.buy_order_id = buy_order_id
        self.sell_order_id = sell_order_id
        self.amount = trade_amount
        self.buy_price = buy_price
        self.sell_price = sell_price
        self.price = trade_price
        
    def __str__(self):
        return '交易ID[%5d] 买单ID[%5d] 卖单ID[%d] 成交数量[%5d] 买单价[%.02f] 卖单价[%.02f] 成交价[%.02f]' % (
            self.id, self.buy_order_id, self.sell_order_id, self.amount, self.buy_price, self.sell_price, self.price)

In [89]:
# 交易撮合引擎（价格优先策略）
class TradeMatchEngine:
    def __init__(self):
        # 买单对列（价格从高到低）
        self.buy_orders = [] 
         # 卖单对列（价格从低到高）
        self.sell_orders = []
        # 撮合成交的交易
        self.match_trades = [] 
        
    # 创建成交
    def _make_trade(self, buy_order, sell_order):
        # 成交价取平均值
        trade_price = (buy_order.price+sell_order.price)/2
        # 成交数量为2者最小值
        trade_amount = min(buy_order.amount, sell_order.amount)
        # 生成成交
        return Trade(buy_order.id, sell_order.id, trade_amount, buy_order.price, sell_order.price, trade_price)
    
    # 撮合买单
    def _handle_buy_order(self, buy_order):
        i = 0
        while i < len(self.sell_orders):
            sell_order = self.sell_orders[i]
            
            # 无法匹配卖单，停止撮合
            if buy_order.price < sell_order.price: 
                break
            
            # 生成成交
            trade = self._make_trade(buy_order, sell_order)
            
            # 扣减数量
            sell_order.amount -= trade.amount
            buy_order.amount -= trade.amount
            
            # 保存交易
            self.match_trades.append(trade)
            
            # 卖单完整撮合?移除队列
            if not sell_order.amount:
                self.sell_orders.pop(i)
            else:
                i += 1
                
            # 买单完整撮合? 停止继续撮合
            if not buy_order.amount:
                break 
        
        # 买单出现部分撮合?加入队列（价格从高到低，相同价格则排在后面）
        if buy_order.amount:
            i = 0
            while i < len(self.buy_orders):
                if buy_order.price > self.buy_orders[i].price:
                    break
                i += 1
            self.buy_orders.insert(i, buy_order)
    
    # 撮合卖单
    def _handle_sell_order(self, sell_order):
        i = 0
        while i < len(self.buy_orders):
            buy_order = self.buy_orders[i]
            
            # 无法匹配买单，停止撮合
            if sell_order.price > buy_order.price:
                break
                
            # 生成成交
            trade = self._make_trade(buy_order, sell_order)
            
            # 扣减数量
            buy_order.amount -= trade.amount
            sell_order.amount -= trade.amount
            
            # 保存交易
            self.match_trades.append(trade)
            
            # 买单完整撮合?移除队列
            if not buy_order.amount:
                self.buy_orders.pop(i)
            else:
                i += 1
                
            # 卖单完整撮合? 停止继续撮合
            if not sell_order.amount:
                break
        
        # 卖单出现部分撮合?加入队列（价格从低到高，相同价格则排在后面）
        if sell_order.amount:
            i = 0
            while i < len(self.sell_orders):
                if sell_order.price < self.sell_orders[i].price:
                    break
                i += 1
            self.sell_orders.insert(i, sell_order)
        
    # API：撮合委托单
    def handle_order(self, order):
        if order.type == Order.BUY:
            self._handle_buy_order(order)
        else:
            self._handle_sell_order(order)
    
    # API：买单列表
    def list_buy_orders(self):
        return self.buy_orders
    
    # API：卖单列表
    def list_sell_orders(self):
        return self.sell_orders
    
    # API：成交列表
    def list_trades(self):
        return self.match_trades

In [90]:
# 测试：交易撮合引擎
engine = TradeMatchEngine()
while True:
    order_type = input('b买 s卖?')
    order_amount = int(input('几手?'))
    order_price = float(input('单价？'))
    if order_type == 'b':
        order_type = Order.BUY
    elif order_type == 's':
        order_type = Order.SELL
    else:
        print('无效操作')
        continue
    order = Order(order_type,order_amount,order_price)
    engine.handle_order(order)
    print('---买单---')
    for order in engine.list_buy_orders():
        print(order)
    print('---卖单---')
    for order in engine.list_sell_orders():
        print(order)
    print('---成交---')
    latest_price = None
    for trade in engine.list_trades():
        print(trade)
        latest_price = trade.price
    print('---行情股价---')
    print(latest_price)

b买 s卖?b
几手?10
单价？1.5
---买单---
订单ID[    1] 操作类型[ buy] 数量[   10] 单价[1.50]
---卖单---
---成交---
---行情股价---
None
b买 s卖?b
几手?15
单价？1.4
---买单---
订单ID[    1] 操作类型[ buy] 数量[   10] 单价[1.50]
订单ID[    2] 操作类型[ buy] 数量[   15] 单价[1.40]
---卖单---
---成交---
---行情股价---
None
b买 s卖?b
几手?20
单价？1.51
---买单---
订单ID[    3] 操作类型[ buy] 数量[   20] 单价[1.51]
订单ID[    1] 操作类型[ buy] 数量[   10] 单价[1.50]
订单ID[    2] 操作类型[ buy] 数量[   15] 单价[1.40]
---卖单---
---成交---
---行情股价---
None
b买 s卖?s
几手?5
单价？1.39
---买单---
订单ID[    3] 操作类型[ buy] 数量[   15] 单价[1.51]
订单ID[    1] 操作类型[ buy] 数量[   10] 单价[1.50]
订单ID[    2] 操作类型[ buy] 数量[   15] 单价[1.40]
---卖单---
---成交---
交易ID[    1] 买单ID[    3] 卖单ID[4] 成交数量[    5] 买单价[1.51] 卖单价[1.39] 成交价[1.45]
---行情股价---
1.45
b买 s卖?b
几手?20
单价？1.5
---买单---
订单ID[    3] 操作类型[ buy] 数量[   15] 单价[1.51]
订单ID[    1] 操作类型[ buy] 数量[   10] 单价[1.50]
订单ID[    5] 操作类型[ buy] 数量[   20] 单价[1.50]
订单ID[    2] 操作类型[ buy] 数量[   15] 单价[1.40]
---卖单---
---成交---
交易ID[    1] 买单ID[    3] 卖单ID[4] 成交数量[    5] 买单价[1.51] 卖单价[1.39] 成交价[1.45]
-

KeyboardInterrupt: Interrupted by user